# Clinical Text Summarization with GPT Model
# ==========================================
This notebook demonstrates how to use natural language processing (NLP)
to create concise summaries of lengthy patient history records.

**INTRODUCTION:**

Healthcare providers are often challenged with reviewing extensive patient
records in limited time. Automatic text summarization can help clinicians
quickly grasp the most important information from a patient's history.
In this assignment, you will implement a system that generates concise
summaries of patient medical records using a language model.


**LEARNING OBJECTIVES:**
- Understand how to apply NLP to healthcare documentation challenges
- Learn effective prompt engineering for medical text summarization
- Implement and evaluate a patient record summarization system
- Identify key clinical information that should be preserved in summaries


In [ ]:
# PART 1: SETUP AND DEPENDENCIES
# Install necessary libraries
# Note: Run this cell to install the required packages for the assignment
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# Import required libraries
# Note: These libraries provide the foundation for working with transformer models
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# PART 2: MODEL SELECTION AND INITIALIZATION
# Use the lightweight GPT-Neo model
# Note: We're using a smaller model for educational purposes. In production,
# you might use a larger model or one specifically fine-tuned for medical text.
model_name = "EleutherAI/gpt-neo-125M"

In [ ]:
# Check for GPU availability
# Note: GPU acceleration significantly speeds up model inference,
# but the code will run on CPU if no GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [ ]:
# Load the tokenizer and model, then move the model to the appropriate device
# Note: The tokenizer converts text to tokens that the model can process,
# while the model itself handles the generation of summaries
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)


# PART 3: SUMMARIZATION FUNCTION
# Note: This function takes a patient history text and returns a concise summary
# STUDENT TASK: Analyze this function carefully to understand how prompt
# engineering affects the quality of the generated summary

def summarize_patient_history(history_text):
    # Construct a prompt that guides the model to generate a summary
    # Note: The prompt is crucial for getting the model to produce the desired output.
    # It provides context and instructions for the summarization task.
    # STUDENT TASK: Consider how you might modify this prompt to improve summary quality.
    # What specific medical elements should the summary prioritize?
    prompt = (
        "Summarize the following 10 years of patient records into a single, concise paragraph. "
        "Highlight critical conditions, treatments, and allergies.\n\n"
        "Patient Records:\n"
        f"{history_text}\n\n"
        "Summary:"
    )

    # Tokenize the prompt with a set maximum token length
    # Note: This converts the text prompt into numeric tokens the model can process.
    # The max_length parameter prevents issues with very long inputs.
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)

    # Generate summary using no_grad to optimize memory usage
    # Note: torch.no_grad() disables gradient calculation, making inference faster
    # and using less memory, which is appropriate for inference tasks.
    # STUDENT TASK: Experiment with different generation parameters to see
    # how they affect the quality and style of the summary.
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_length=256,          # Adjust based on desired summary length
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the generated output
    # Note: This converts the model's output tokens back to human-readable text
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract text after the prompt's "Summary:" marker
    # Note: This isolates just the summary part from the full generated text
    summary = full_text.split("Summary:")[-1].strip()
    return summary

In [ ]:
# PART 4: TESTING WITH EXAMPLE DATA
# Example synthetic patient history for demonstration
# Note: In a real-world scenario, this data would come from electronic health records
# STUDENT TASK: Create additional test cases with different patient profiles to evaluate
# how well the system handles various medical conditions and history patterns
patient_history = """
Patient Name: John Doe
Age: 58
Medical Records:
2010: Diagnosed with type 2 diabetes. Prescribed metformin.
2011: High blood pressure noted; started on lisinopril.
2013: Experienced mild heart arrhythmia; advised regular check-ups.
2015: Underwent minor surgery for gallbladder removal.
2017: Allergic reaction to penicillin reported.
2018: Diabetes management improved.
2020: Incident of stroke; received thrombolytic therapy.
2022: Follow-up indicated partial recovery.
2023: Routine check-up: cholesterol levels high; advised dietary changes.
2024: Continued management of diabetes and hypertension.
"""

In [ ]:
# Generate and display the summary
# Note: This runs the summarization function and displays the results
# STUDENT TASK: Evaluate the quality of this summary. Does it capture all
# critical medical information? What's missing or unnecessarily included?
summary = summarize_patient_history(patient_history)
print("Summary of Patient History:")
print(summary)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Summary of Patient History:
The author is currently conducting a clinical trial to evaluate the efficacy of the following medications:

Atypical medications, such as metformin, are considered to be effective for the prevention of type 2 diabetes. These medications may reduce the risk of developing type 2 diabetes by approximately 5% to 10% (as the medication


# PART 5: ASSIGNMENT TASKS
TODO: Extend this notebook to:
1. Improve the prompt to better extract critical clinical information
2. Create an evaluation function that measures summary quality:
    - Coverage of key medical events
    - Inclusion of critical allergies and conditions
    - Appropriate length and readability
3. Implement a function to handle multiple patient records efficiently
4. Add post-processing to ensure sensitive patient information is properly handled
5. Compare summaries generated with different prompts and model parameters
6. Create a method to highlight potential gaps or inconsistencies in patient records